In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install beautifulsoup4
!pip3 install --upgrade pip

In [2]:
import pandas as pd
import requests
import numpy as np 
import time
from bs4 import BeautifulSoup
import string
import csv

/Users/arhan/Desktop/ML_ufc/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:

# Function to update highest score
def update_highest_score(score, highest_score):
    # Ensure highest_score reflects the best score achieved so far
    if score > highest_score:
        highest_score = score
    return highest_score

# Load factor data from CSV (static_data.csv)
def load_factor_data(filename):
    factor_data = {}
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            fighter_name = row['name'].strip()
            factor_data[fighter_name] = float(row['Gfactor'])
    return factor_data
# Load factor data once at the start
factor_data = load_factor_data('static_data.csv')
# Function to get factor for a given opponent's name
def get_factor(opponent_name):
    return factor_data.get(opponent_name, 1)  # Default to 1 if not found
# Determines the style of victory multiplier based on the way of victory
def get_style_of_victory(way_of_fight):     
    if way_of_fight in ["KO", "TKO", "SUB"]:
        return 1.1
    elif way_of_fight == "U-DEC":
        return 1.0
    elif way_of_fight == "M-DEC":
        return 0.9
    elif way_of_fight == "S-DEC":
        return 0.8
    elif way_of_fight == "DQ":
        return 0
    else:
        return 1.0  # Default for cases like Draw or No Contest


#___________________________________
# ADD THE LINK OF THE FIGHTER HERE |
#------------------------------------
fighter_url = "http://www.ufcstats.com/fighter-details/1338e2c7480bdf9e"

response = requests.get(fighter_url)
soup = BeautifulSoup(response.text, 'html.parser')
score = 0  # Reset fighter score for each fighter
highest_score = -1000  # Initialize highest score

# Extract the fighter's name
name_tag = soup.find('span', class_='b-content__title-highlight')
fighter_name = name_tag.text.strip() if name_tag else "Unknown Fighter"
# Find all fight rows (from newest to oldest)
fight_rows = soup.select('.b-fight-details__table-row')[::-1]
# Loop through each fight
for row in fight_rows:
    # Extract W/L (Win/Loss/Next/NC/Draw)
    W_L_element = row.select_one('.b-flag__text')
    if W_L_element:
        W_L_text = W_L_element.text.strip().lower()
        if W_L_text == "win":
            W_L = "WIN"
        elif W_L_text == "loss":
            W_L = "LOSS"
        else:
            continue  # Ignore Next, NC, Draw
    else:
        continue  # If no flag is found, skip to the next row

    # Extract the opponent name (second fighter)
    opponent_name = row.select('td.b-fight-details__table-col.l-page_align_left p.b-fight-details__table-text a.b-link.b-link_style_black')[1].text.strip()

    # Extract way_of_fight (KO, TKO, U-DEC, S-DEC, etc.)
    way_of_fight = row.select('td.b-fight-details__table-col.l-page_align_left p.b-fight-details__table-text')[4].text.strip()
    
    # Clean way_of_fight to only contain relevant data (e.g., "KO/TKO" or "U-DEC")
    if "KO" in way_of_fight or "TKO" in way_of_fight:
        way_of_fight = "KO/TKO"
    elif "SUB" in way_of_fight:
        way_of_fight = "SUB"
    elif "U-DEC" in way_of_fight:
        way_of_fight = "U-DEC"
    elif "M-DEC" in way_of_fight:
        way_of_fight = "M-DEC"
    elif "S-DEC" in way_of_fight:
        way_of_fight = "S-DEC"
    elif "DQ" in way_of_fight:
        way_of_fight = "DQ"
    else:
        way_of_fight = None

    # Calculate Gfactor for opponent
    opponent_gfactor = get_factor(opponent_name)

    # Determine style of victory multiplier
    style_of_victory = get_style_of_victory(way_of_fight)

    # Update fighter score based on win/loss
    if W_L == 'WIN':
        score += 10 * (opponent_gfactor / 100) * style_of_victory
    elif W_L == 'LOSS':
        score -= (10 * style_of_victory)
    else:
        continue  # Continue for cases like Draw, Next, etc.

    # Update highest score after each fight
    highest_score = update_highest_score(score, highest_score)

    print(f"Fighter: {fighter_name} |   Score: {score} | Peak Score: {highest_score}   | Opponent: {opponent_name}")







Fighter: Israel Adesanya |   Score: 8.776234253165141 | Peak Score: 8.776234253165141   | Opponent: Rob Wilkinson
Fighter: Israel Adesanya |   Score: 16.402708853410623 | Peak Score: 16.402708853410623   | Opponent: Marvin Vettori
Fighter: Israel Adesanya |   Score: 21.350631590560667 | Peak Score: 21.350631590560667   | Opponent: Brad Tavares
Fighter: Israel Adesanya |   Score: 29.832309156643976 | Peak Score: 29.832309156643976   | Opponent: Derek Brunson
Fighter: Israel Adesanya |   Score: 37.93700945407706 | Peak Score: 37.93700945407706   | Opponent: Anderson Silva
Fighter: Israel Adesanya |   Score: 44.38876109438465 | Peak Score: 44.38876109438465   | Opponent: Kelvin Gastelum
Fighter: Israel Adesanya |   Score: 56.851575239160546 | Peak Score: 56.851575239160546   | Opponent: Robert Whittaker
Fighter: Israel Adesanya |   Score: 63.75076484023513 | Peak Score: 63.75076484023513   | Opponent: Yoel Romero
Fighter: Israel Adesanya |   Score: 73.53657576995803 | Peak Score: 73.53657